# Least Square

In [2]:
import sys
my_path = r'/home/ilaria/Scrivania/Machine_Learning/Project_1/Project1_ML'
sys.path.insert(0,my_path + r'/code/COMMON')

import numpy as np 
import matplotlib.pyplot as plt
from proj1_helpers import * 
from implementations import *
from outliers import handle_outliers
from labels import idx_2labels
from standard import standardize
from costs import compute_loglikelihood_reg
from sigmoid import * 
from extend_features import *

In [3]:
yb, input_data, ids = load_csv_data(my_path + r'/data/train.csv', sub_sample=False)
print('Data loaded!')
print(input_data.shape)

Data loaded!
(250000, 30)


# Handle -999

In [6]:
input_data, Y = handle_outliers(input_data,yb,-999,0) # substiution with mean because the standardization
                                                           #can be affected, otherwise we should delete the whole row
ind_back, ind_sig = idx_2labels(Y, [-1,1])
Y[ind_back] = 0

# get feature names 
names = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])
log = True
degree = 3
X0, features = extend_features(input_data, names, degree, log)

-999 are replaced by 0
---------------------------
Features have been set to the power(s): [1, 2, 3]
4 logarithmic features have been added.
Data have been standardized.
---------------------------


In [8]:
X0.shape

(250000, 94)

# Features

In [9]:

idx = [3, 13, 39, 4, 40, 52, 21, 22, 33, 68, 57, 73, 6, 48, 38, 41, 23, 50, 43, 91, 5, 66, 82, 9, 64, 78, 18, 0, 1, 2, 90, 34, 12, 14, 67, 93, 69, 92, 79, 29, 88, 87, 35, 37, 15, 31, 30, 28, 32, 16, 36, 63, 24, 70, 19, 20] 
X = X0[:,idx]



# Set the parameters

In [5]:
initial_w = np.ones(X.shape[1])
max_iters = 50
gamma = 10e-6
method = 'gd'

In [10]:
w, loss = least_squares(Y,X)

In [11]:
y = predict_labels(w,X)

In [12]:
y.shape

(250000,)

In [13]:
pos = 0
neg = 0
for i in range (len(Y)):
    if Y[i] == y[i]:
        pos += 1
    else:
        neg += 1
print(pos)
print(neg)

success_rate = pos/(pos+neg)
print(success_rate)

172165
77835
0.68866


# Load the data test

In [15]:
nope, test_data, ids = load_csv_data(r'/home/ilaria/Scrivania/Machine_Learning/Project_1/test.csv', sub_sample=False)

print("Data loaded! Shape: ")
print(np.shape(test_data))

Data loaded! Shape: 
(568238, 30)


In [16]:
input_data, Y = handle_outliers(test_data,nope,-999,0)
ind_back, ind_sig = idx_2labels(Y, [-1,1])
Y[ind_back] = 0

# get feature names 
names = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])
log = True
degree = 3
X0, features = extend_features(input_data, names, degree, log)
idx = [3, 13, 39, 4, 40, 52, 21, 22, 33, 68, 57, 73, 6, 48, 38, 41, 23, 50, 43, 91, 5, 66, 82, 9, 64, 78, 18, 0, 1, 2, 90, 34, 12, 14, 67, 93, 69, 92, 79, 29, 88, 87, 35, 37, 15, 31, 30, 28, 32, 16, 36, 63, 24, 70, 19, 20] 
X = X0[:,idx]

y_pred = predict_labels(w,X)

-999 are replaced by 0
---------------------------
Features have been set to the power(s): [1, 2, 3]
4 logarithmic features have been added.
Data have been standardized.
---------------------------


In [17]:
y_pred.shape

(568238,)

In [18]:
y_pred[np.where(y_pred==0)] = -1

In [19]:
create_csv_submission(ids, y_pred, "third_sub")